In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
import tensorflow.python.keras as keras
from tensorflow.python.keras.layers import Layer
import tensorflow as tf
import dan_models
import dan_utils

In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
randseed = 25
dan_utils.setup_seed(randseed)
res = 11

v = pd.read_csv('../data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('../res/%i_res%i_id_402_withclass.csv'%(randseed, res), index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = det_with_class['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

seg = pd.read_csv('../data/segement.csv', header=None)
num_dets = 25

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = dan_utils.get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp)
    v_class_temp = v_class_temp[v_class_temp['id'].isin(det_list_class_temp[:num_dets])]
    v_class[i] = v_class_temp
    
near_road_set = []
for i in range(num_class):
    near_road_set.append(dan_utils.rds_mat(dist_mat, det_list_class[i][:num_dets], seg))

There are 5 class(es)


C:\Users\10169\anaconda3\envs\dan_traff\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [5]:
# ind, class
# 0  , blue
# 1  , green
# 2  , yellow  <--
# 3  , black   <--
# 4  , red     <--
class_color_set = ['b', 'g', 'y', 'black', 'r']
class_i = 3

near_road = np.array(near_road_set[class_i])
flow = v_class[class_i].iloc[:, 2:-1]

prop = 24/31  # proportion of training data

k, t_p, t_input, t_pre, num_links = 5, 10, 12, 3, 25
from_day = 1
to_day = 31

image_train, image_test, day_train, day_test, label_train, label_test\
= dan_utils.sliding_window(
    flow, near_road, from_day, to_day, prop, 
    k, t_p, t_input, t_pre, num_links
)

(25, 4464)
(1161, 5, 12, 25)
(898, 5, 12, 25)
(263, 5, 12, 25)
(898, 25, 3)
(263, 25, 3)


In [6]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

finish_model = dan_models.build_model(input_data, input_HA)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 5, 12, 25)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 5, 12, 25)    100         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 5, 12, 25)    5650        batch_normalization[0][0]        
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 5, 12, 25)    0           conv2d[0][0]                     
______________________________________________________________________________________________

In [7]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [9]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=100, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Epoch 1/100
8/8 [==============================] - 5s 307ms/step - loss: 3096.2844 - val_loss: 2798.3850
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 3019.5224 - val_loss: 2554.9436
Epoch 3/100
8/8 [==============================] - 0s 10ms/step - loss: 2941.1755 - val_loss: 2227.6953
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 2776.3560 - val_loss: 1867.8177
Epoch 5/100
8/8 [==============================] - 0s 11ms/step - loss: 2532.2979 - val_loss: 1540.4291
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 2175.2804 - val_loss: 1470.9834
Epoch 7/100
8/8 [==============================] - 0s 10ms/step - loss: 1738.9518 - val_loss: 1654.0148
Epoch 8/100
8/8 [==============================] - 0s 9ms/step - loss: 1426.2293 - val_loss: 1470.9597
Epoch 9/100
8/8 [==============================] - 0s 10ms/step - loss: 1237.0168 - val_loss: 1471.1746
Epoch 10/100
8/8 [==============================] - 0s 10ms/step 

8/8 [==============================] - 0s 10ms/step - loss: 306.1393 - val_loss: 274.8107
Epoch 81/100
8/8 [==============================] - 0s 10ms/step - loss: 309.1649 - val_loss: 271.0162
Epoch 82/100
8/8 [==============================] - 0s 10ms/step - loss: 294.5230 - val_loss: 269.4679
Epoch 83/100
8/8 [==============================] - 0s 10ms/step - loss: 284.6088 - val_loss: 267.2784
Epoch 84/100
8/8 [==============================] - 0s 10ms/step - loss: 280.1802 - val_loss: 262.4241
Epoch 85/100
8/8 [==============================] - 0s 9ms/step - loss: 281.6650 - val_loss: 255.9054
Epoch 86/100
8/8 [==============================] - 0s 10ms/step - loss: 273.1882 - val_loss: 249.1166
Epoch 87/100
8/8 [==============================] - 0s 9ms/step - loss: 276.4010 - val_loss: 249.2533
Epoch 88/100
8/8 [==============================] - 0s 10ms/step - loss: 263.8241 - val_loss: 251.9217
Epoch 89/100
8/8 [==============================] - 0s 10ms/step - loss: 256.7260 - val_

In [10]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [11]:
#计算各项误差指标

mape_mean = dan_utils.mape_loss_func(model_pre, label_test)
smape_mean = dan_utils.smape_loss_func(model_pre, label_test)
mae_mean = dan_utils.mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.25527158840778263
smape = 0.2526023363547699
mae = 10.605989407584948


In [12]:
#模型保存
finish_model.save_weights('../model/source_%s.h5'%class_color_set[class_i])


In [13]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = dan_utils.mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.22997356120509624
2th link
0.1935355415922355
3th link
0.11628256403840749
4th link
0.21029477585455417
5th link
0.2773035945893709
6th link
0.23348698552527372
7th link
0.1892325250766825
8th link
0.17038652602353632
9th link
0.18105823836615886
10th link
0.33118764985647986
11th link
0.10421944331455532
12th link
0.24808171562886341
13th link
0.32739189994105183
14th link
0.20607212768361394
15th link
0.2210488032095213
16th link
0.10487537230031485
17th link
0.18475625512552563
18th link
0.2202433417491724
19th link
0.1853833935788615
20th link
0.37369617565313734
21th link
0.22524423708841385
22th link
0.27253148998603216
23th link
0.3194893970558492
24th link
0.9381974300535522
25th link
0.3178166656983058


In [14]:
mape_pd = pd.Series(mape_list)

In [15]:
mape_pd.sort_values()

10    0.104219
15    0.104875
2     0.116283
7     0.170387
8     0.181058
16    0.184756
18    0.185383
6     0.189233
1     0.193536
13    0.206072
3     0.210295
17    0.220243
14    0.221049
20    0.225244
0     0.229974
5     0.233487
11    0.248082
21    0.272531
4     0.277304
24    0.317817
22    0.319489
12    0.327392
9     0.331188
19    0.373696
23    0.938197
dtype: float64